In [304]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

In [305]:
df = pd.read_csv("data_data_berita_detik (1).csv")

In [306]:
df.head()

,judul,tanggal,isi,url,kategori
0,Boston Celtics dan Ironi Kyrie Irving,"Selasa, 18 Jun 2024 13:45 WIB",Jakarta - Boston Celtics jadi juara NBA 2024. ...,https://sport.detik.com/basket/d-7396168/bosto...,Olahraga
1,"Boston Celtics Juara NBA 2024, Gelar ke-18","Selasa, 18 Jun 2024 12:15 WIB",Jakarta - Boston Celtics mengalahkan Dallas Ma...,https://sport.detik.com/basket/d-7396066/bosto...,Olahraga
2,Titik Nadir Khamzat Chimaev,"Selasa, 18 Jun 2024 11:15 WIB","Jakarta - Sempat jadi kejutan di UFC, selanjut...",https://sport.detik.com/sport-lain/d-7395950/t...,Olahraga
3,"Situasi Sulit Joan Mir: Mau Tinggalkan Honda, ...","Senin, 17 Jun 2024 20:30 WIB",Jakarta - Joan Mir belum mendapatkan tim untuk...,https://sport.detik.com/moto-gp/d-7395407/situ...,Olahraga
4,Eng Hian Sebut Olimpiade 2024 Lebih Berat dari...,"Senin, 17 Jun 2024 18:20 WIB",Jakarta - Pelatih ganda putri Eng Hian menyebu...,https://sport.detik.com/raket/d-7395283/eng-hi...,Olahraga


In [307]:
def clean_text(text):
    text = text.lower()
    text = text.replace(",", "")
    text = text.replace('"', "")
    text = text.replace("-", "")
    text = text.replace(".", "")
    text = text.replace(":", "")
    text = text.replace(")", "")
    text = text.replace("(", "")
    text = text.replace("/", "")
    text = text.replace("\n", "")
    return text

In [322]:
df['kategori'].unique()

array(['Olahraga', 'Kesehatan', 'Pariwisata', 'Pendidikan', 'Otomotif',
       'Kuliner', 'Keuangan'], dtype=object)

In [308]:
df.drop(columns=['tanggal', 'url'], inplace=True)
df['isi'] = df['isi'].apply(clean_text)
df['judul'] = df['judul'].apply(clean_text)
df['content'] = df['judul'] + " " + df['isi']

In [309]:
X = df['content']
y = df['kategori']

In [310]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [311]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [312]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [313]:
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_pad = pad_sequences(X_train_seq, padding="post", maxlen=50)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, padding="post", maxlen=50)

In [314]:
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=16)) # Mengubah token menjadi vektor yang bisa di mengerti neural network
model.add(GlobalAveragePooling1D()) # Mengambil rata rata dari semua kata
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(len(df['kategori'].unique()), activation="softmax"))

In [315]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [316]:
model.fit(X_train_pad, y_train_cat, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
175/175 [==============================] - 1s 947us/step - loss: 1.7127 - accuracy: 0.3486
Epoch 2/100
175/175 [==============================] - 0s 947us/step - loss: 0.7709 - accuracy: 0.7798
Epoch 3/100
175/175 [==============================] - 0s 913us/step - loss: 0.4546 - accuracy: 0.8546
Epoch 4/100
175/175 [==============================] - 0s 987us/step - loss: 0.3435 - accuracy: 0.8912
Epoch 5/100
175/175 [==============================] - 0s 977us/step - loss: 0.2778 - accuracy: 0.9130
Epoch 6/100
175/175 [==============================] - 0s 1ms/step - loss: 0.2275 - accuracy: 0.9355
Epoch 7/100
175/175 [==============================] - 0s 930us/step - loss: 0.1956 - accuracy: 0.9400
Epoch 8/100
175/175 [==============================] - 0s 949us/step - loss: 0.1649 - accuracy: 0.9536
Epoch 9/100
175/175 [==============================] - 0s 988us/step - loss: 0.1428 - accuracy: 0.9582
Epoch 10/100
175/175 [==============================] - 0s 944us/step - los

In [317]:
loss, accuracy = model.evaluate(X_test_pad, y_test_cat)
prediction = model.predict(X_test_pad)
print(prediction)
prediction_classes = np.argmax(prediction, axis=1)
decoded_labels = encoder.inverse_transform(prediction_classes)
print("Hasil Prediksi: ", decoded_labels)

44/44 [==============================] - 0s 641us/step
[[4.2823476e-24 1.0000000e+00 1.5833036e-17 ... 6.2239690e-14
  7.6874018e-27 9.5949978e-15]
 [4.4301977e-17 2.5756860e-18 3.4772473e-18 ... 4.9061323e-20
  1.0000000e+00 1.2377262e-11]
 [4.7836145e-31 1.0637116e-29 1.4575721e-26 ... 4.5546334e-35
  1.0000000e+00 3.7187978e-11]
 ...
 [1.5702383e-26 1.0000000e+00 3.9531932e-14 ... 5.2993504e-21
  6.8879729e-27 2.0154962e-10]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 6.5273597e-24]
 [4.5182467e-25 4.8742833e-11 8.1922045e-24 ... 1.0000000e+00
  1.4003790e-14 7.6311224e-21]]
Hasil Prediksi:  ['Keuangan' 'Pariwisata' 'Pariwisata' ... 'Keuangan' 'Olahraga' 'Otomotif']


In [318]:
def getData(kata):
    input_data = {
    "content": [
            kata
        ]
    }
    input_data_df = pd.DataFrame(input_data)
    input_data_df["content"] = input_data_df["content"].apply(clean_text)
    input_data_seq = tokenizer.texts_to_sequences(input_data_df["content"])
    input_data_pad = pad_sequences(input_data_seq, padding="post", maxlen=50)
    return input_data_pad

def klasifikasi(text):
    prediction = model.predict(getData(text))
    prediction_classes = np.argmax(prediction, axis=1)
    decoded_labels = encoder.inverse_transform(prediction_classes)
    print("Hasil Klasifikasi:", decoded_labels[0])

In [319]:
klasifikasi("Menteri pendidikan meminta untuk dimulainya pelajaran coding dari Sekolah Dasar")

1/1 [==============================] - 0s 14ms/step
Hasil Klasifikasi: Keuangan


In [321]:
model.save("model.h5")
joblib.dump(tokenizer, "tokenizer.joblib")
joblib.dump(pad_sequences, "padded.joblib")
joblib.dump(encoder, "encoder.joblib")

['encoder.joblib']